<a href="https://colab.research.google.com/github/Ryoma-HCU/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pandasを用いた課題プログラムの作成

In [ ]:
print("こんにちは")